In [1]:
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import string

In [2]:
data = pd.read_csv('analysisData.csv',low_memory=False)

In [3]:
# data_fillna = data.copy()
# data_fillna['transit'] = data['transit'].fillna('far')
# data = data_fillna

In [4]:
data_nona = data[(data['transit'].notnull())]
data = data_nona

In [5]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def stem(doc):
    return [porter.stem(word) for word in word_tokenize(doc.lower())
           if word not in set(string.punctuation)]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(stop_words='english',tokenizer=stem,max_features=1000)
X = vectorizer.fit_transform(data['transit'])
features = vectorizer.get_feature_names()
kmeans = KMeans(2)
kmeans.fit(X)
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
print("top features for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(features[i] for i in centroid)))

top features for each cluster:
0: minut, walk, train, min, manhattan, 5, 10, away, subway, stop
1: train, block, subway, away, line, street, bu, park, station, stop


In [7]:
from collections import Counter
Counter(kmeans.predict(X))

Counter({1: 12949, 0: 7206})

In [8]:
assigned_cluster = kmeans.transform(X).argmin(axis=1)
for i in range(kmeans.n_clusters):
    cluster = np.arange(0, X.shape[0])[assigned_cluster==i]
    sample_articles = np.random.choice(cluster, 3, replace=False)
    print("cluster %d:" % i)
    for article in sample_articles:
        print("    %s" % data.loc[article]['transit'])

cluster 0:
    The house is a 4 mn walk to Grand L Station.
    Very close to a quick train ride to Manhattan on either the 2, 3, 4, 5, A, or C lines. 15 minutes to Downtown Manhattan, 40 minutes to Central Park. 40 minutes from JFK airport. Car service also available by phone
    Very close to L, G Trains at the Lorimer/Metropolitan stop.  Also very close to the J,M trains on the Hewes stop.  15 to 20 minute commute to the Lower East Side in Manhattan.
cluster 1:
    The apartment is a short walk to the 7 subway line, and provides quick access to Mid-town Manhattan, Grand Central Station, and Times Square.  Also there is a quick bus directly into the Greenpoint and Williamsburg Brooklyn--the B24.


KeyError: 'the label [10989] is not in the [index]'

In [37]:
cluster_mapping = {
    0: 'medium',
    1: 'far',
    2: 'close',
}

In [51]:
data['transit_category'] = kmeans.predict(X)
data['transit_category'] = data['transit_category'].apply(lambda x: cluster_mapping.get(x))


In [52]:
data['transit']

0         close
1        medium
2        medium
3           far
4           far
5           far
6         close
7           far
8         close
9         close
10          far
11       medium
12       medium
13        close
14          far
15       medium
16        close
17          far
18          far
19        close
20          far
21       medium
22          far
23        close
24       medium
25       medium
26        close
27          far
28       medium
29        close
          ...  
29112       far
29113     close
29114    medium
29115       far
29116     close
29117     close
29118     close
29119     close
29120     close
29121    medium
29122     close
29123     close
29124     close
29125       far
29126     close
29127       far
29128     close
29129     close
29130     close
29131    medium
29132     close
29133    medium
29134    medium
29135    medium
29136       far
29137    medium
29138     close
29139       far
29140    medium
29141     close
Name: transit, Length: 2